In [1]:
import pandas as pd
import numpy as np
import pandas as pd
import glob

# 동대문구 14개 행정동 리스트
dong_list = [
    "용신동", "제기동", "전농1동", "전농2동", "답십리1동", "답십리2동",
    "장안1동", "장안2동", "청량리동", "회기동", "휘경1동", "휘경2동",
    "이문1동", "이문2동"
]

# 모든 CSV 파일 경로 수집
file_paths = glob.glob('RAW/*단속 위치정보*.csv')

filtered_dfs = []
for file in file_paths:
    # CSV 읽기 (인코딩 자동 감지)
    try:
        df = pd.read_csv(file, encoding='cp949')
    except:
        df = pd.read_csv(file, encoding='utf-8')
    # '구주소' 열에서 동 이름이 포함된 행 필터링
    df['구주소'] = df['구주소'].astype(str)
    mask = df['구주소'].apply(lambda addr: any(dong in addr for dong in dong_list))
    filtered_dfs.append(df[mask])

# 모든 행정동 데이터 병합
merged_df = pd.concat(filtered_dfs, ignore_index=True)

# 결과 저장
output_path = 'PROCESSED/dongdaemun_all_dongs.csv'
merged_df.to_csv(output_path, index=False)

In [2]:
import pandas as pd
import glob

# 동대문구 14개 행정동 리스트
dong_list = [
    "용신동", "제기동", "전농1동", "전농2동", "답십리1동", "답십리2동",
    "장안1동", "장안2동", "청량리동", "회기동", "휘경1동", "휘경2동",
    "이문1동", "이문2동"
]

# 모든 신고현황 CSV 파일 경로 수집
file_paths = glob.glob('RAW/*불법주정차 신고현황*.csv')

# 필터링된 데이터프레임을 담을 리스트
filtered_dfs = []

for path in file_paths:
    # CSV 파일 읽기 (인코딩 자동 감지)
    try:
        df = pd.read_csv(path, encoding='cp949')
    except UnicodeDecodeError:
        df = pd.read_csv(path, encoding='utf-8')
    
    # '구주소' 열에서 동 이름 포함 여부로 필터링ㅠ
    df['주소'] = df['주소'].astype(str)
    mask = df['주소'].apply(lambda addr: any(dong in addr for dong in dong_list))
    filtered_dfs.append(df[mask])

# 병합
merged_reports = pd.concat(filtered_dfs, ignore_index=True)

# 결과를 CSV로 저장 (원하는 경로로 수정 가능)
merged_reports.to_csv('PROCESSED/dongdaemun_reports_merged.csv', index=False)


In [3]:
merged_reports

,민원접수일,민원접수시간,주소,경도,위도
0,20211028,18:19:00,동대문구 회기로 24 (지번) 제기동 1144-3 삼성정육점식당 앞 종암대교 상,127.039162,37.591097
1,20211028,22:08:00,서울 동대문구 회기동 103-270,127.049112,37.590539
2,20211029,11:18:00,동대문구 경동시장로12길 43 (제기동 473-2),127.043088,37.582254
3,20211029,11:18:00,동대문구 왕산로35길 64 (제기동 476),127.042871,37.582536
4,20211029,15:46:00,동대문구 회기동 3-16,127.054429,37.591870
...,...,...,...,...,...
4580,20240313,10:45:00,서울특별시 동대문구 제기동 268-37,127.033248,37.581047
4581,20240313,11:46:00,서울 동대문구 회기동 77-38,127.052468,37.590414
4582,20240313,12:38:00,동대문구 고산자로62길 6(제기동 136-256)앞 무인.정카페부근 코너 불법주정차,127.037091,37.589219
4583,20240313,13:53:00,서울특별시 동대문구 회기동 42-41,127.051576,37.592709


In [4]:
merged_df['단속일'] = pd.to_datetime(merged_df['단속일'], format='%Y%m%d', errors='coerce').dt.date
merged_df['단속일'] = pd.to_datetime(merged_df['단속일'])
merged_reports['민원접수일'] = pd.to_datetime(merged_reports['민원접수일'], format='%Y%m%d', errors='coerce').dt.date
merged_reports['민원접수일'] = pd.to_datetime(merged_reports['민원접수일'])

In [6]:
import pandas as pd

# 1. 동대문구 14개 행정동 리스트
dong_list = [
    "용신동", "제기동", "전농1동", "전농2동", "답십리1동", "답십리2동",
    "장안1동", "장안2동", "청량리동", "회기동", "휘경1동", "휘경2동",
    "이문1동", "이문2동"
]

# 2. CSV 파일 읽기 (인코딩 문제 발생 시 cp949→utf-8 순으로 시도)
def read_csv_auto(path):
    try:
        return pd.read_csv(path, encoding='cp949')
    except UnicodeDecodeError:
        return pd.read_csv(path, encoding='utf-8')

df_enf = read_csv_auto('PROCESSED/dongdaemun_all_dongs.csv')          # 단속 데이터
df_rep = read_csv_auto('PROCESSED/dongdaemun_reports_merged.csv')    # 신고(민원) 데이터

# 3. 상세 주소에서 행정동만 추출하는 함수
def extract_dong(addr):
    for dong in dong_list:
        if dong in str(addr):
            return dong
    return None

df_enf['행정동'] = df_enf['구주소'].apply(extract_dong)
df_rep['행정동'] = df_rep['주소'].apply(extract_dong)

# 5. 일자별·행정동별 건수 집계
enf_counts = (
    df_enf
    .groupby(['단속일', '행정동'])
    .size()
    .reset_index(name='단속건수')
    .rename(columns={'단속일':'일자'})
)

rep_counts = (
    df_rep
    .groupby(['민원접수일', '행정동'])
    .size()
    .reset_index(name='신고건수')
    .rename(columns={'민원접수일':'일자'})
)

# 6. 두 집계 결과를 병합(outer join) 및 결손값 0으로 채우기
merged = pd.merge(
    rep_counts,
    enf_counts,
    on=['일자', '행정동'],
    how='outer'
)
merged['신고건수'] = merged['신고건수'].fillna(0).astype(int)
merged['단속건수'] = merged['단속건수'].fillna(0).astype(int)

# 7. 정렬 및 인덱스 리셋
merged = merged.sort_values(['일자', '행정동']).reset_index(drop=True)

# 8. 결과 확인 및 저장
print(merged.head())
merged.to_csv('PROCESSED/dongdaemun_daily_summary.csv', index=False)

         일자    행정동  신고건수  단속건수
0  20210929  답십리1동     0    17
1  20210929  답십리2동     0     5
2  20210929   이문1동     0     3
3  20210929   이문2동     0     1
4  20210929   장안1동     0     6
